 1. Data transformation (log, hot encoding, nan)
 
 2. Testing SVM regression models
 Applying log transformation really increases the accuracy.

Removing columns with more than 1000 NaNs gives better result than applying "mean" to them

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import Imputer

from sklearn import svm

import warnings
warnings.filterwarnings('ignore')

from subprocess import check_output
print(check_output(["ls", "/home/wang/Desktop/Auberon/paper/1_CCVAE"]).decode("utf8"))

connect to sql server ip 54.ipynb
CVAE
CVAE.html
CVAE.ipynb
feature_selection
lightGBM
patent_dataset_0001percent.csv
patent_top500_2013.csv
SVR
top500_companies_relationship_file.pkl



In [2]:
train = pd.read_csv('/home/wang/Desktop/Auberon/paper/1_CCVAE/CVAE/train_data.csv')
labels=train["TimeCitedFiveYearCNT"]
test = pd.read_csv('/home/wang/Desktop/Auberon/paper/1_CCVAE/CVAE/test_data.csv')
data = pd.concat([train,test],ignore_index=True)
data = data.drop("TimeCitedFiveYearCNT", 1)
ids = test["PatentNumber"]

In [6]:
train.head()

,PatentNumber,TimeCitedFiveYearCNT,ForeignRefCNT,SUM_For_distance_5Y,ReferenceCNT,GeneralityFiveYear5Class_cpc,GeneralityFiveYear5Class,GeneralityTenYear5Class,Generality5Class,GeneralityFiveYear4Class,...,Originality5Class_cpc,InvCtry_Generality_5Y,CPCCNT,InventorCNT,Originality4Class,Originality4Class_cpc,AssAss_Cultural_Distance,InvInvDistance,Originality3Class_cpc,InventorCountryCNT
0,8350903,0.0,5.0,13.7556,5.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.720000,0.00,14.0,2,0.800000,0.480000,0.0,12.64846,0.480000,1
1,8600431,0.0,8.0,0.0000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,3.0,2,0.000000,0.000000,0.0,211.79955,0.000000,1
2,8358150,9.0,22.0,212673.2446,1103.0,0.781065,0.915556,0.915556,0.915556,0.791111,...,0.995586,0.32,14.0,2,0.971011,0.962405,0.0,42.89113,0.806227,1
3,8532449,3.0,0.0,300761.1173,7.0,0.666667,0.444444,0.444444,0.444444,0.000000,...,0.816327,0.00,6.0,2,0.816327,0.734694,0.0,909.99111,0.612245,1
4,8589228,0.0,0.0,0.0000,3.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,2.0,5,0.666667,0.000000,0.0,3233.20451,0.000000,2


In [12]:
# Count the number of rows in train
train.shape[0]


107804

In [13]:
# Count the number of rows in total
data.shape[0]

134755

In [14]:
# Count the number of NaNs each column has.
nans=pd.isnull(data).sum()
nans[nans>0]

Series([], dtype: int64)

In [7]:
#將欄位處理掉(if needs)
# data=data.drop("Id", 1)
# data=data.drop("Alley", 1)
# data=data.drop("Fence", 1)
# data=data.drop("MiscFeature", 1)
# data=data.drop("PoolQC", 1)
# data=data.drop("FireplaceQu", 1)

In [15]:
# Count the column types
data.dtypes.value_counts()

float64    30
int64       4
dtype: int64

In [16]:
data.head()

,PatentNumber,ForeignRefCNT,SUM_For_distance_5Y,ReferenceCNT,GeneralityFiveYear5Class_cpc,GeneralityFiveYear5Class,GeneralityTenYear5Class,Generality5Class,GeneralityFiveYear4Class,GeneralityTenYear4Class,...,Originality5Class_cpc,InvCtry_Generality_5Y,CPCCNT,InventorCNT,Originality4Class,Originality4Class_cpc,AssAss_Cultural_Distance,InvInvDistance,Originality3Class_cpc,InventorCountryCNT
0,8350903,5.0,13.7556,5.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.720000,0.00,14.0,2,0.800000,0.480000,0.0,12.64846,0.480000,1
1,8600431,8.0,0.0000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,3.0,2,0.000000,0.000000,0.0,211.79955,0.000000,1
2,8358150,22.0,212673.2446,1103.0,0.781065,0.915556,0.915556,0.915556,0.791111,0.791111,...,0.995586,0.32,14.0,2,0.971011,0.962405,0.0,42.89113,0.806227,1
3,8532449,0.0,300761.1173,7.0,0.666667,0.444444,0.444444,0.444444,0.000000,0.000000,...,0.816327,0.00,6.0,2,0.816327,0.734694,0.0,909.99111,0.612245,1
4,8589228,0.0,0.0000,3.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,2.0,5,0.666667,0.000000,0.0,3233.20451,0.000000,2


In [10]:
#找出類別型資料(if needs)
# all_columns = data.columns.values
# non_categorical = ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", 
#                    "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", 
#                    "2ndFlrSF", "LowQualFinSF", "GrLivArea", "GarageArea", 
#                    "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", 
#                    "ScreenPorch","PoolArea", "MiscVal"]

# categorical = [value for value in all_columns if value not in non_categorical]

In [22]:
# One Hot Encoding and nan transformation
data = pd.get_dummies(data)

imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
data = imp.fit_transform(data)

# Log transformation
data = np.log(data)
labels = np.log(labels)

# Change -inf to 0 again
data[data==-np.inf]=0

In [23]:
# Split traing and test
train = data[:107804]
test = data[107804:]

In [24]:
train.shape

(107804, 34)

In [25]:
labels.shape

(107804,)

In [26]:
clf = svm.SVR(kernel="rbf")

In [27]:
clf.fit(train,labels)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [17]:
test_labels = clf.predict(test)

In [18]:
test_labels.shape

(1459,)

In [19]:
print(test_labels)

[ 11.81624666  12.23470153  12.14372613 ...,  12.08087383  11.60034954
  12.44674734]


In [20]:
price_label = np.exp(test_labels)
price_labels = price_label.reshape(-1,)

In [21]:
print(price_labels)

[ 135434.9400385   205808.52634403  187911.45262461 ...,  176464.29286235
  109135.94024166  254421.42909117]


In [22]:
sub = pd.DataFrame({
        "SalePrice": price_labels
    })

In [23]:
sub.head()

,SalePrice
0,135434.940039
1,205808.526344
2,187911.452625
3,202696.372009
4,179084.218149


In [24]:
test_data = pd.read_csv('../input/test.csv')

In [25]:
result =  test_data.append(sub)

In [26]:
result['SalePrice'] = sub['SalePrice']

In [27]:
result['SalePrice'].head()

0    135434.940039
1    205808.526344
2    187911.452625
3    202696.372009
4    179084.218149
Name: SalePrice, dtype: float64

In [28]:
result.to_csv("sample_submission.csv", index=False)

In [32]:
import xgboost as xgb